<a href="https://colab.research.google.com/github/carlosperz88/dasany/blob/main/Generaci%C3%B3n_de_Im%C3%A1genes_con_IA_(Stable_Diffusion%2C_etc_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caja de Herramientas para Diseños Printful (Colab + Gradio)

Este cuaderno combina varias herramientas útiles para preparar tus diseños para Printful, todo dentro de una interfaz gráfica (Gradio) con pestañas:

1.  **Removedor de Fondo:** Elimina el fondo de tu diseño (usa `rembg`).
2.  **Creador de Mockups:** Superpone tu diseño (con fondo transparente) sobre una plantilla.
3.  **Ajustes de Imagen:** Modifica brillo, contraste y saturación. Convierte a escala de grises.
4.  **Añadir Texto:** Agrega texto simple a tu imagen.
5.  **Upscaler Básico (Opcional/Experimental):** Intenta mejorar la resolución (usa `Pillow` - **Nota:** Para upscaling de alta calidad con IA como Real-ESRGAN, se necesitaría una configuración más compleja y librerías adicionales, lo cual puede exceder los límites de Colab gratuito fácilmente. Esta versión usa un método simple).

**Instrucciones:**
1.  **Instalar Librerías:** Ejecuta la primera celda de código (`Instalación`). Puede tardar unos minutos.
2.  **Iniciar Interfaz:** Ejecuta la segunda celda de código (`Aplicación Gradio`). Espera a que aparezca el enlace `Running on public URL: ...` y haz clic en él.
3.  **Usar la Interfaz:** Navega por las diferentes pestañas para usar cada herramienta. Sube las imágenes necesarias y ajusta los controles.
4.  **Descargar:** En cada pestaña, haz clic derecho sobre la imagen de salida para guardarla.
5.  **¡Importante!** Este cuaderno consume muchos recursos. Si encuentras errores de memoria, ve a `Entorno de ejecución` -> `Reiniciar entorno de ejecución` y vuelve a ejecutar las celdas. Considera usar las herramientas una a la vez.

In [2]:
print("Instalando librerías requeridas...")
# -q para modo silencioso
!pip install rembg[gpu] gradio Pillow -q
print("Instalando paquete de fuentes (Liberation)...")
# Instala fuentes comunes como Liberation Sans (similar a Arial)
!apt-get update -qq && apt-get install -y fonts-liberation -qq
print("Instalaciones completadas.")

# Verificar instalaciones (opcional)
try:
    import rembg
    import gradio
    from PIL import Image, ImageFont
    print("Librerías básicas (rembg, gradio, Pillow) instaladas correctamente.")
    # Verificar si la fuente está accesible
    try:
        # Usa una ruta común para Liberation Sans Regular
        font_path_check = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
        _ = ImageFont.truetype(font_path_check, size=10)
        print(f"Fuente de prueba ({font_path_check}) cargada exitosamente.")
    except IOError:
        print(f"ADVERTENCIA: No se pudo cargar la fuente de prueba {font_path_check}. El texto usará la fuente por defecto.")
except ImportError as e:
    print(f"Error importando librerías básicas: {e}")

print("\nNota: La primera ejecución de RemBG descargará modelos (puede tardar).")

Instalando librerías requeridas...
Instalando paquete de fuentes (Liberation)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Instalaciones completadas.
Librerías básicas (rembg, gradio, Pillow) instaladas correctamente.
Fuente de prueba (/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf) cargada exitosamente.

Nota: La primera ejecución de RemBG descargará modelos (puede tardar).


In [3]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import io
import os
import numpy as np # Necesario para algunas operaciones de imagen/Gradio

# --- 1. Funciones de Ayuda (con correcciones para transparencia y texto) ---

# 1a. Removedor de Fondo (Sin cambios funcionales, pero asegura salida RGBA)
from rembg import remove as remove_bg_rembg

def remove_background_func(input_image_pil):
    if input_image_pil is None:
        gr.Warning("Por favor, sube una imagen primero.")
        return None
    print("Removiendo fondo...")
    try:
        input_image_pil = input_image_pil.convert("RGBA") # Asegurar RGBA entrada
        byte_arr = io.BytesIO()
        input_image_pil.save(byte_arr, format='PNG')
        input_bytes = byte_arr.getvalue()
        output_bytes = remove_bg_rembg(input_bytes)
        output_image = Image.open(io.BytesIO(output_bytes)).convert("RGBA") # Asegurar RGBA salida
        print("Fondo removido exitosamente.")
        return output_image
    except Exception as e:
        print(f"Error en remove_background_func: {e}")
        gr.Error(f"Error al quitar fondo: {e}")
        return None

# 1b. Creador de Mockups (con DEBUG y asegura RGBA)
def create_mockup_func(background_img_pil, foreground_img_pil, pos_x, pos_y, scale):
    if background_img_pil is None:
        gr.Info("Por favor, sube una imagen de fondo (plantilla).")
        return None

    background = background_img_pil.convert("RGBA") # Asegurar RGBA fondo

    if foreground_img_pil is None:
         gr.Info("Sube una imagen de diseño (primer plano) para superponer.")
         return background # Devolver fondo si no hay diseño

    # --- DEBUG ---
    print(f"DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: {foreground_img_pil.mode}")
    # --- FIN DEBUG ---

    print("Creando mockup...")
    try:
        # Forzar conversión a RGBA del diseño por si acaso
        foreground = foreground_img_pil.convert("RGBA")
        print(f"DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: {foreground.mode}")

        original_width, original_height = foreground.size
        new_width = max(1, int(original_width * scale))
        new_height = max(1, int(original_height * scale))
        foreground_resized = foreground.resize((new_width, new_height), Image.Resampling.LANCZOS)

        combined_image = background.copy()
        # Pegar usando la máscara alfa del diseño redimensionado
        combined_image.paste(foreground_resized, (pos_x, pos_y), foreground_resized)
        print("Mockup creado.")
        return combined_image # Devolver como RGBA

    except Exception as e:
        print(f"Error en create_mockup_func: {e}")
        gr.Error(f"Error al crear mockup: {e}")
        return background # Devolver fondo original en caso de error grave


# 1c. Ajustes de Imagen (asegura RGBA)
def adjust_image_func(input_image_pil, brightness, contrast, saturation, grayscale):
    if input_image_pil is None:
        gr.Warning("Sube una imagen para ajustar.")
        return None
    print("Ajustando imagen...")
    try:
        img = input_image_pil.copy().convert("RGBA") # Trabajar en RGBA
        alpha = img.split()[-1]
        img_rgb = img.convert("RGB")
        if grayscale:
            img_adjusted_rgb = ImageEnhance.Color(img_rgb).enhance(0)
        else:
            img_temp = img_rgb
            enhancer = ImageEnhance.Brightness(img_temp)
            img_temp = enhancer.enhance(brightness)
            enhancer = ImageEnhance.Contrast(img_temp)
            img_temp = enhancer.enhance(contrast)
            enhancer = ImageEnhance.Color(img_temp)
            img_adjusted_rgb = enhancer.enhance(saturation)
        # Volver a juntar con el canal alfa
        img_final = Image.merge("RGBA", (*img_adjusted_rgb.split(), alpha))
        print("Ajuste completado.")
        return img_final
    except Exception as e:
        print(f"Error en adjust_image_func: {e}")
        gr.Error(f"Error al ajustar imagen: {e}")
        return None

# 1d. Añadir Texto (con fuente TTF y tamaño)
def add_text_func(input_image_pil, text_to_add, font_size, pos_x, pos_y, color_hex):
    if input_image_pil is None:
        gr.Warning("Sube una imagen para añadirle texto.")
        return None
    if not text_to_add:
         gr.Info("Escribe el texto que quieres añadir.")
         return input_image_pil
    print("Añadiendo texto...")
    try:
        img = input_image_pil.copy().convert("RGBA") # Trabajar en RGBA
        draw = ImageDraw.Draw(img)

        # Intentar cargar fuente Liberation Sans (instalada en Celda 2)
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
        try:
            font = ImageFont.truetype(font_path, size=int(font_size)) # Usar tamaño
            print(f"Fuente '{font_path}' cargada con tamaño {font_size}.")
        except IOError:
            print(f"ADVERTENCIA: No se pudo cargar '{font_path}'. Usando fuente PIL por defecto (tamaño fijo).")
            font = ImageFont.load_default()
            gr.Warning("No se pudo cargar la fuente TTF. Usando fuente básica sin tamaño ajustable.")
        except Exception as e:
             print(f"Error inesperado cargando fuente: {e}. Usando fuente por defecto.")
             font = ImageFont.load_default()
             gr.Warning(f"Error cargando fuente: {e}. Usando fuente básica.")


        # Convertir color hexadecimal a tupla RGBA
        try:
            color_hex = color_hex.lstrip('#')
            rgb = tuple(int(color_hex[i:i+2], 16) for i in (0, 2, 4))
            fill_color = rgb + (255,)
        except ValueError:
            gr.Warning("Color hexadecimal inválido, usando negro.")
            fill_color = (0, 0, 0, 255)

        print(f"Dibujando texto '{text_to_add}' en ({pos_x},{pos_y}) con tamaño {font_size}")
        draw.text((pos_x, pos_y), text_to_add, font=font, fill=fill_color)
        print("Texto añadido.")
        return img # Devolver como RGBA
    except Exception as e:
        print(f"Error en add_text_func: {e}")
        gr.Error(f"Error al añadir texto: {e}")
        return input_image_pil.convert("RGBA") if input_image_pil else None

# 1e. Upscaler (Simple - Pillow, asegura RGBA si entrada es RGBA)
def upscale_simple_func(input_image_pil, scale_factor):
    if input_image_pil is None:
        gr.Warning("Sube una imagen para ampliar.")
        return None
    print(f"Ampliando imagen x{scale_factor} (método simple)...")
    try:
        # Preservar modo original (ej: RGBA)
        original_mode = input_image_pil.mode
        width, height = input_image_pil.size
        new_width = max(1, int(width * scale_factor))
        new_height = max(1, int(height * scale_factor))
        upscaled_img = input_image_pil.resize((new_width, new_height), Image.Resampling.LANCZOS)
        print("Ampliación simple completada.")
        # Asegurarse de que la salida tenga el modo original si se perdió
        if upscaled_img.mode != original_mode:
             print(f"WARN: Modo cambiado durante upscale. Forzando a {original_mode}")
             upscaled_img = upscaled_img.convert(original_mode)
        return upscaled_img
    except Exception as e:
        print(f"Error en upscale_simple_func: {e}")
        gr.Error(f"Error en upscaling simple: {e}")
        return None

# --- Función auxiliar para el flujo ---
def pass_through(data):
    if data is None:
        gr.Info("La imagen de origen está vacía.")
        return None
    print(f"Pasando imagen al siguiente paso (tipo: {type(data)}, modo: {getattr(data, 'mode', 'N/A')})")
    # Asegurarse de que siempre se pasa como PIL.Image (Gradio a veces puede cambiar el tipo)
    if not isinstance(data, Image.Image):
         try:
             # Intentar convertir si es posible (ej: numpy array)
             data = Image.fromarray(data)
             print("Convertido data a PIL.Image en pass_through")
         except Exception as e:
              print(f"Error convirtiendo data a PIL en pass_through: {e}")
              gr.Error("Error interno al pasar la imagen.")
              return None
    # Forzar RGBA al pasar entre etapas puede ser útil
    # return data.convert("RGBA")
    return data # Devolver tal cual por ahora

# --- 2. Crear la Interfaz Gradio (con format="png" y slider de tamaño) ---

print("Creando la interfaz Gradio v4 (Transparencia y Texto mejorados)...")

with gr.Blocks(theme=gr.themes.Default()) as demo: # Probar tema Default
    gr.Markdown("# Caja de Herramientas para Diseños Printful v4")
    gr.Markdown("Usa las pestañas y los botones '⬇️ Usar Salida...' para procesar tu imagen.")

    with gr.Tabs():

        # Pestaña 1: Removedor de Fondo
        with gr.TabItem("1. Removedor de Fondo") as tab1:
            gr.Markdown("Sube una imagen para quitarle el fondo.")
            with gr.Row():
                bg_remover_input = gr.Image(type="pil", label="1. Imagen Original")
                # Forzar PNG en salida
                bg_remover_output = gr.Image(type="pil", label="Salida Sin Fondo", format="png")
            bg_remover_button = gr.Button("Quitar Fondo")
            send_to_mockup_btn = gr.Button("⬇️ Usar Salida Sin Fondo en Mockup (Diseño)")

        # Pestaña 2: Creador de Mockups
        with gr.TabItem("2. Creador de Mockups") as tab2:
            gr.Markdown("Superpón el diseño sobre la plantilla.")
            with gr.Row():
                with gr.Column():
                    mockup_bg_input = gr.Image(type="pil", label="Plantilla de Fondo")
                    mockup_fg_input = gr.Image(type="pil", label="Diseño a Superponer (PNG Transparente)")
                with gr.Column():
                    # Forzar PNG en salida
                    mockup_output = gr.Image(type="pil", label="Salida Mockup", format="png")
            with gr.Row():
                mockup_pos_x = gr.Slider(minimum=-500, maximum=2500, step=5, value=50, label="Posición X")
                mockup_pos_y = gr.Slider(minimum=-500, maximum=2500, step=5, value=50, label="Posición Y")
                mockup_scale = gr.Slider(minimum=0.05, maximum=5.0, step=0.05, value=0.5, label="Escala Diseño")
            send_to_adjust_btn = gr.Button("⬇️ Usar Salida Mockup en Ajustes")

        # Pestaña 3: Ajustes de Imagen
        with gr.TabItem("3. Ajustes de Imagen") as tab3:
             gr.Markdown("Ajusta brillo, contraste, saturación o convierte a escala de grises.")
             with gr.Row():
                adjust_input = gr.Image(type="pil", label="Imagen a Ajustar")
                # Forzar PNG en salida
                adjust_output = gr.Image(type="pil", label="Salida Ajustada", format="png")
             with gr.Row():
                adjust_brightness = gr.Slider(minimum=0.1, maximum=3.0, step=0.05, value=1.0, label="Brillo")
                adjust_contrast = gr.Slider(minimum=0.1, maximum=3.0, step=0.05, value=1.0, label="Contraste")
                adjust_saturation = gr.Slider(minimum=0.0, maximum=3.0, step=0.05, value=1.0, label="Saturación")
             adjust_grayscale = gr.Checkbox(label="Convertir a Escala de Grises")
             send_to_text_btn = gr.Button("⬇️ Usar Salida Ajustada en Añadir Texto")

        # Pestaña 4: Añadir Texto
        with gr.TabItem("4. Añadir Texto") as tab4:
             gr.Markdown("Añade texto usando fuente Liberation Sans (instalada).")
             with gr.Row():
                text_input_img = gr.Image(type="pil", label="Imagen para Añadir Texto")
                # Forzar PNG en salida
                text_output_img = gr.Image(type="pil", label="Salida con Texto", format="png")
             text_input_text = gr.Textbox(label="Texto a añadir", placeholder="Escribe aquí...")
             # Reintroducir slider de tamaño
             text_font_size = gr.Slider(minimum=8, maximum=250, step=1, value=50, label="Tamaño de Fuente")
             with gr.Row():
                text_pos_x = gr.Slider(minimum=0, maximum=2000, step=1, value=20, label="Posición X Texto")
                text_pos_y = gr.Slider(minimum=0, maximum=2000, step=1, value=20, label="Posición Y Texto")
             text_color = gr.ColorPicker(label="Color del Texto", value="#000000")
             send_to_upscale_btn = gr.Button("⬇️ Usar Salida con Texto en Upscaler")

        # Pestaña 5: Upscaler Simple (Pillow)
        with gr.TabItem("5. Upscaler (Simple)") as tab5:
            gr.Markdown("**Experimental:** Aumenta tamaño con redimensionado LANCZOS.")
            with gr.Row():
                upscale_input = gr.Image(type="pil", label="Imagen a Ampliar")
                # Forzar PNG en salida
                upscale_output = gr.Image(type="pil", label="Salida Ampliada", format="png")
            upscale_factor = gr.Slider(minimum=1.1, maximum=4.0, step=0.1, value=2.0, label="Factor de Escala")
            upscale_button = gr.Button("Ampliar Imagen (Simple)")
            gr.Markdown("---")
            gr.Markdown("**FIN DEL FLUJO:** Guarda esta imagen final.")


    # --- Conectar Lógica y Flujo ---

    # Tab 1 Lógica y Flujo
    bg_remover_button.click(remove_background_func, inputs=bg_remover_input, outputs=bg_remover_output)
    send_to_mockup_btn.click(pass_through, inputs=bg_remover_output, outputs=mockup_fg_input)

    # Tab 2 Lógica y Flujo
    mockup_inputs = [mockup_bg_input, mockup_fg_input, mockup_pos_x, mockup_pos_y, mockup_scale]
    for inp in mockup_inputs:
        inp.change(create_mockup_func, inputs=mockup_inputs, outputs=mockup_output)
    send_to_adjust_btn.click(pass_through, inputs=mockup_output, outputs=adjust_input)

    # Tab 3 Lógica y Flujo
    adjust_inputs = [adjust_input, adjust_brightness, adjust_contrast, adjust_saturation, adjust_grayscale]
    for inp in adjust_inputs:
        inp.change(adjust_image_func, inputs=adjust_inputs, outputs=adjust_output)
    send_to_text_btn.click(pass_through, inputs=adjust_output, outputs=text_input_img)

    # Tab 4 Lógica y Flujo (ahora incluye font_size)
    text_inputs_for_func = [text_input_img, text_input_text, text_font_size, text_pos_x, text_pos_y, text_color]
    text_update_triggers = [text_input_img, text_input_text, text_font_size, text_pos_x, text_pos_y, text_color]
    for inp in text_update_triggers:
         inp.change(add_text_func, inputs=text_inputs_for_func, outputs=text_output_img)
    send_to_upscale_btn.click(pass_through, inputs=text_output_img, outputs=upscale_input)

    # Tab 5 Lógica
    upscale_button.click(upscale_simple_func, inputs=[upscale_input, upscale_factor], outputs=upscale_output)

    # --- Mensajes Finales ---
    gr.Markdown("---")
    gr.Markdown("**Consejos:** Haz clic derecho en las imágenes de salida para guardarlas. Usa los botones '⬇️' para el flujo.")

# --- 3. Lanzar la Interfaz Gradio ---

print("Lanzando la interfaz Gradio v4...")
print("Espera a que aparezca el enlace 'Running on public URL:' y haz clic en él.")

demo.launch(share=True, debug=True)

Creando la interfaz Gradio v4 (Transparencia y Texto mejorados)...
Lanzando la interfaz Gradio v4...
Espera a que aparezca el enlace 'Running on public URL:' y haz clic en él.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cb64c7381bbfd9d659.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Removiendo fondo...
Fondo removido exitosamente.
Pasando imagen al siguiente paso (tipo: <class 'PIL.Image.Image'>, modo: RGB)
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: RGBA
Mockup creado.
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: RGBA
Mockup creado.
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: RGBA
Mockup creado.
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: RGBA
Mockup creado.
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo de imagen de diseño DESPUÉS de convertir: RGBA
Mockup creado.
DEBUG: Mockup - Modo de imagen de diseño RECIBIDA: RGB
Creando mockup...
DEBUG: Mockup - Modo